In [1]:
import os
import datetime
import geopandas as gpd
import pandas as pd
import json
import shapely.ops

import sentinelhub
import boto3

/Users/nikhilsrajan/NASA-Harvest/project/fetch_satdata/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append('..')

In [3]:
import config

import cdseutils.utils as utils
import cdseutils.mydataclasses as mydataclasses
import cdseutils.constants as constants
import cdseutils.sentinel2 as sentinel2

import mysecrets
import s2l1c_via_s3

In [4]:
cdse_creds = mysecrets.PROFILES['nikhilsasirajan@gmail.com']

In [5]:
utils.cdse_credentials_to_json(cdse_creds=cdse_creds, filepath='../data/cdse_credentials.json')

In [6]:
_cdse_creds = utils.cdse_credentials_from_json(filepath='../data/cdse_credentials.json')

In [7]:
_cdse_creds.sh_creds == cdse_creds.sh_creds, _cdse_creds.s3_creds == cdse_creds.s3_creds

(True, True)

In [8]:
year = 2021

startdate = datetime.datetime(year, 3, 1)
enddate = datetime.datetime(year + 1, 1, 31)

In [9]:
(enddate - startdate).days

336

In [10]:
catalog_filepath = '../data/eth_bb_east_s2l1c_catalog.geojson'
# roi_filepath = '../../ethiopia/data/Ethiopia_Bounding_Boxs/Ethiopia_2021_East_Bounding_Box.shp'
roi_filepath = '../../ethiopia/data/Ethiopia_Bounding_Boxs/Ethiopia_2019_North_Bounding_Box.shp'
# roi_filepath = '../data/test_eth_east_bb_s2grids.geojson',
eth_s2grids_filepath = '../../ethiopia/data/ethiopia_s2_grids_esa_stats.geojson'

In [11]:
# catalog_gdf = gpd.read_file(catalog_filepath)

In [12]:
eth_s2grids_gdf = gpd.read_file(eth_s2grids_filepath)

In [13]:
s2grid_of_interest = '165bca4'

In [14]:
selected_s2grid_gdf = eth_s2grids_gdf[
    eth_s2grids_gdf['id'] == s2grid_of_interest
][['id','geometry']]
selected_s2grid_gdf

,id,geometry
26458,165bca4,"POLYGON ((36.21969 11.68559, 36.26308 11.67928..."


In [15]:
selected_s2grid_gdf.to_file('../data/s2grid_165bca4.geojson')

In [16]:
# shapes_gdf = gpd.read_file(roi_filepath)
# bboxes = utils.get_bboxes(shapes_gdf=shapes_gdf)
bboxes = utils.get_bboxes(shapes_gdf=selected_s2grid_gdf)
catalog_gdf, results = utils.fetch_catalog(
    bboxes = bboxes,
    sh_creds = cdse_creds.sh_creds,
    collection = sentinelhub.DataCollection.SENTINEL2_L1C,
    startdate = startdate,
    enddate = enddate,
    cache_folderpath = config.FOLDERPATH_FETCH_CATALOG_CACHE,
)

/Users/nikhilsrajan/NASA-Harvest/project/fetch_satdata/.venv/lib/python3.10/site-packages/sentinelhub/constants.py:114: SHUserWarning: sentinelhub-py supports only WGS 84 coordinate reference system with coordinate order lng-lat. Given pyproj.CRS(4326) has coordinate order lat-lng. Be careful to use the correct order of coordinates.
  warnings.warn(message, category=SHUserWarning)


In [17]:
len(results)

516

In [18]:
results

[{'stac_version': '1.0.0',
  'stac_extensions': ['https://stac-extensions.github.io/eo/v1.0.0/schema.json',
   'https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
  'id': 'S2A_MSIL1C_20220129T080201_N0400_R035_T36PZT_20220129T095350.SAFE',
  'type': 'Feature',
  'geometry': {'type': 'MultiPolygon',
   'crs': {'type': 'name',
    'properties': {'name': 'urn:ogc:def:crs:OGC::CRS84'}},
   'coordinates': [[[[35.7521608838858, 11.74676257433028],
      [36.47588293916897, 11.738866188147462],
      [36.2470776823771, 10.750046210388085],
      [35.742752986565776, 10.75491403664618],
      [35.7521608838858, 11.74676257433028]]]]},
  'bbox': [35.742752986565776,
   10.750046210388085,
   36.47588293916897,
   11.74676257433028],
  'properties': {'datetime': '2022-01-29T08:16:54.183Z',
   'platform': 'sentinel-2a',
   'instruments': ['msi'],
   'constellation': 'sentinel-2',
   'gsd': 10,
   'eo:cloud_cover': 0.0,
   'proj:epsg': 32636,
   'proj:bbox': [799980.0, 1190220.0, 9

In [21]:
s3_creds = cdse_creds.s3_creds
s3url = 's3://EODATA/Sentinel-2/MSI/L1C_N0500/2021/03/02/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652.SAFE/'

In [22]:
s3 = boto3.resource(
    's3',
    endpoint_url = s3_creds.endpoint_url,
    aws_access_key_id = s3_creds.s3_access_key,
    aws_secret_access_key = s3_creds.s3_secret_key,
    region_name = s3_creds.region_name,
)

root_s3path = utils.s3url_to_s3path(s3url=s3url)

all_files = s3.Bucket(root_s3path.bucket).objects.filter(Prefix=root_s3path.prefix)

In [24]:
all_files = list(all_files)

In [26]:
xml_s3paths = [mydataclasses.S3Path(bucket=file.bucket_name, prefix=file.key) for file in all_files if file.key.endswith('.xml')]

In [27]:
xml_s3paths

[S3Path(bucket='EODATA', prefix='Sentinel-2/MSI/L1C_N0500/2021/03/02/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652.SAFE/DATASTRIP/DS_S2RP_20230523T123652_S20210302T075213/MTD_DS.xml'),
 S3Path(bucket='EODATA', prefix='Sentinel-2/MSI/L1C_N0500/2021/03/02/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652.SAFE/DATASTRIP/DS_S2RP_20230523T123652_S20210302T075213/QI_DATA/FORMAT_CORRECTNESS.xml'),
 S3Path(bucket='EODATA', prefix='Sentinel-2/MSI/L1C_N0500/2021/03/02/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652.SAFE/DATASTRIP/DS_S2RP_20230523T123652_S20210302T075213/QI_DATA/GENERAL_QUALITY.xml'),
 S3Path(bucket='EODATA', prefix='Sentinel-2/MSI/L1C_N0500/2021/03/02/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652.SAFE/DATASTRIP/DS_S2RP_20230523T123652_S20210302T075213/QI_DATA/GEOMETRIC_QUALITY.xml'),
 S3Path(bucket='EODATA', prefix='Sentinel-2/MSI/L1C_N0500/2021/03/02/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652.SAFE/DATASTRIP

In [28]:
for s3path in xml_s3paths:
    utils.download_s3_file(
        s3_creds = cdse_creds.s3_creds,
        s3path = s3path,
        download_folderpath = '../data/xml_files/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652'
    )

---------------------
File set to download:
---------------------
bucket: EODATA
prefix: Sentinel-2/MSI/L1C_N0500/2021/03/02/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652.SAFE/DATASTRIP/DS_S2RP_20230523T123652_S20210302T075213/MTD_DS.xml
---------------------
---------------------
---------------------
File set to download:
---------------------
bucket: EODATA
prefix: Sentinel-2/MSI/L1C_N0500/2021/03/02/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652.SAFE/DATASTRIP/DS_S2RP_20230523T123652_S20210302T075213/QI_DATA/FORMAT_CORRECTNESS.xml
---------------------
---------------------
---------------------
File set to download:
---------------------
bucket: EODATA
prefix: Sentinel-2/MSI/L1C_N0500/2021/03/02/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652.SAFE/DATASTRIP/DS_S2RP_20230523T123652_S20210302T075213/QI_DATA/GENERAL_QUALITY.xml
---------------------
---------------------
---------------------
File set to download:
---------------------
bucket

In [30]:
import xml.etree.ElementTree as ET

In [31]:
root = ET.parse('../data/xml_files/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652/MTD_TL.xml').getroot()

In [36]:
root

<Element '{https://psd-14.sentinel2.eo.esa.int/PSD/S2_PDI_Level-1C_Tile_Metadata.xsd}Level-1C_Tile_ID' at 0x30bd12660>

In [68]:
element_mean_sun_angle = root.findall('.//Mean_Sun_Angle/AZIMUTH_ANGLE')[0]
element_mean_sun_angle.text, element_mean_sun_angle.attrib['unit']

('126.343836473649', 'deg')

In [67]:
root.findall('.//Mean_Sun_Angle/AZIMUTH_ANGLE')[0].attrib['unit']

'deg'

In [ ]:
{x['properties']['proj:epsg'] for x in results}

In [ ]:
{x['properties']['proj:epsg'] for x in results}

In [ ]:
catalog_gdf[:2]

In [ ]:
successful_download_count, total_download_count \
= s2l1c_via_s3.download_sentinel2_l1c_tiles(
    cdse_creds = cdse_creds,
    catalog_gdf = catalog_gdf[:2],
    sentinel2_local_catalog_filepath = config.FILEPATH_SENTINEL2_LOCAL_CATALOG,
    catalog_save_folderpath = config.FOLDERPATH_FETCH_CATALOG_CACHE,
    root_download_folderpath = config.FOLDERPATH_SATELLITE,
    roi_filepath = roi_filepath,
    startdate = startdate,
    enddate = enddate,
    upper_limit_for_number_of_tiles = 50,
    # chunksize_for_download_and_update_catalog = 100,
)

In [ ]:
local_catalog_gdf = gpd.read_file(config.FILEPATH_SENTINEL2_LOCAL_CATALOG)

In [ ]:
local_catalog_gdf